In [137]:
import os
os.chdir('C:\\Users\\M246047\\Documents\\Python')
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
from datetime import datetime, timedelta
import re
import tensorflow as tf
import keras
from zipfile import ZipFile
import cv2

# Preprocessing/Cleaning Libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
import scipy.stats as stats

# Import various componenets for model building
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop

# Dimensionality Reduction & Unsupervised Learning
from sklearn.decomposition import PCA
from sklearn.manifold import LocallyLinearEmbedding, TSNE
import umap
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn import datasets, metrics
from sklearn.neural_network import BernoulliRBM
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import linkage as lnkg
from sklearn.neural_network import MLPClassifier


# Import the backend
from keras import backend as K
from keras.layers import Activation, Dropout, Flatten, Dense

In [7]:
!pip install opencv-python

In [4]:
import cv2

In [111]:
image = cv2.imread('8863_idx5_x51_y1251_class0.png', 0)
print(type(image))
image.shape

<class 'numpy.ndarray'>


(50, 50)

In [97]:
x = image.reshape((1,) + image.shape)
print(x.shape)
x

(1, 50, 50)


array([[[247, 246, 247, ..., 131, 109, 117],
        [246, 246, 246, ..., 136, 130, 139],
        [246, 246, 246, ..., 133, 125, 113],
        ...,
        [246, 246, 246, ..., 241, 243, 246],
        [246, 246, 246, ..., 246, 244, 244],
        [246, 246, 245, ..., 242, 245, 244]]], dtype=uint8)

In [ ]:

for img in os.listdir('./raw_data'):
#    imgName = img.split('.')[0] # converts '0913209.jpg' --> '0913209'
#     label = naming_dict[str(imgName)]
#     counting_dict[label] += 1
    path = os.path.join('./raw_data', img)
    saveName = './labeled_train/' + label + '-' + str(counting_dict[label]) + '.jpg'
    image_data = np.array(Image.open(path))
    imageio.imwrite(saveName, image_data)

In [51]:


print("Current Working Directory " , os.getcwd())
os.chdir('8864')
print("Current Working Directory " , os.getcwd())

Current Working Directory  C:\Users\M246047\Documents\Python\8864


FileNotFoundError: [WinError 2] The system cannot find the file specified: '8864'

In [64]:
# Testing image loading from an individual folder
import os
entries = os.listdir('C:\\Users\\M246047\\Documents\\Python\8864')
df = []

for folder in entries: 
    print(str(folder))
    for image in folder:
        image_name = str(image)
        print(image_name)
        image_data = cv2.imread(image, 0)
        print(image_data)
        df[image] = image_data
        

0
0
None


TypeError: list indices must be integers or slices, not str

In [114]:

image = cv2.imread('C:\\Users\\M246047\\Documents\\Python\\8864\\8863_idx5_x51_y1251_class0.png', 0)
print(type(image))
image.shape

<class 'NoneType'>


AttributeError: 'NoneType' object has no attribute 'shape'

In [112]:
image = cv2.imread('8863_idx5_x51_y1251_class0.png', 0)
print(type(image))
image.shape

<class 'numpy.ndarray'>


(50, 50)

In [118]:
image_data = cv2.imread('C:\\Users\\M246047\\Documents\\Python\\8864\\0\\8864_idx5_x1001_y1501_class0.png', 0)
print(image_data)

[[219 232 241 ... 235 242 242]
 [217 238 242 ... 241 238 237]
 [227 235 241 ... 224 217 234]
 ...
 [239 239 240 ... 241 241 240]
 [240 241 241 ... 237 241 240]
 [240 240 239 ... 233 241 241]]


In [196]:
images_0_8863 = []
for root, dirs, files in os.walk('C:\\Users\\M246047\\Documents\\Python\\8863\\0'):
    for filename in files:
        image_data = cv2.imread('C:\\Users\\M246047\\Documents\\Python\\8863\\0\\'+filename, 0)
#         print(filename)
#         print(image_data.shape)
        if image_data.shape==(50,50):
#             image_data.reshape(1,50,50)
            images_0_8863.append(image_data)
            
images_0_8863 = np.asarray(images_0_8863)    
print('There are {} images'.format(len(images_0_8863)))
print('The type of the data is', type(images_0_8863))
print('The shape of the data is', images_0_8863.shape)
images_0_8863_class = np.zeros((len(images_0_8863),), dtype=int)

There are 768 images
The type of the data is <class 'numpy.ndarray'>
The shape of the data is (768, 50, 50)


In [193]:
# Creating a list of image names
images_1_8863 = []
for root, dirs, files in os.walk('C:\\Users\\M246047\\Documents\\Python\\8863\\1'):
    for filename in files:
        image_data = cv2.imread('C:\\Users\\M246047\\Documents\\Python\\8863\\1\\'+filename, 0)
#         print(filename)
#         print(image_data.shape)
        if image_data.shape==(50,50):
#             image_data.reshape(1,50,50)
            images_1_8863.append(image_data)
            
images_1_8863 = np.asarray(images_1_8863)
print('There are {} images'.format(len(images_1_8863)))
print('The type of the data is', type(images_1_8863))
print('The shape of the data is', images_1_8863.shape)
images_1_8863_class = np.ones((len(images_1_8863),), dtype=int)

There are 207 images
The type of the data is <class 'numpy.ndarray'>
The shape of the data is (207, 50, 50)


In [204]:
images_0_8864 = []
for root, dirs, files in os.walk('C:\\Users\\M246047\\Documents\\Python\\8864\\0'):
    for filename in files:
        image_data = cv2.imread('C:\\Users\\M246047\\Documents\\Python\\8864\\0\\'+filename, 0)
#         print(filename)
#         print(image_data.shape)
        if image_data.shape==(50,50):
#             image_data.reshape(1,50,50)
            images_0_8864.append(image_data)
            
images_0_8864 = np.asarray(images_0_8864)    
print('There are {} images'.format(len(images_0_8864)))
print('The type of the data is', type(images_0_8864))
images_0_8864_class = np.zeros((len(images_0_8864)), dtype=int)

There are 730 images
The type of the data is <class 'numpy.ndarray'>


In [202]:
# Creating a list of image names
images_1_8864 = []
for root, dirs, files in os.walk('C:\\Users\\M246047\\Documents\\Python\\8864\\1'):
    for filename in files:
        image_data = cv2.imread('C:\\Users\\M246047\\Documents\\Python\\8864\\1\\'+filename, 0)
#         print(filename)
#         print(image_data.shape)
        if image_data.shape==(50,50):
#              image_data.reshape(1,50,50)
            images_1_8864.append(image_data)
            
images_1_8864 = np.asarray(images_1_8864)
print('There are {} images'.format(len(images_1_8864)))
print('The type of the data is', type(images_1_8864))
images_1_8864_class = np.ones((len(images_1_8864),), dtype=int)

There are 326 images
The type of the data is <class 'numpy.ndarray'>


In [205]:
images_0_8865 = []
for root, dirs, files in os.walk('C:\\Users\\M246047\\Documents\\Python\\8865\\0'):
    for filename in files:
        image_data = cv2.imread('C:\\Users\\M246047\\Documents\\Python\\8865\\0\\'+filename, 0)
#         print(filename)
#         print(image_data.shape)
        if image_data.shape==(50,50):
            image_data.reshape(1,50,50)
            images_0_8865.append(image_data)
            
images_0_8865 = np.asarray(images_0_8865)    
print('There are {} images'.format(len(images_0_8865)))
print('The type of the data is', type(images_0_8865))
images_0_8865_class = np.zeros((len(images_0_8865)), dtype=int)

There are 657 images
The type of the data is <class 'numpy.ndarray'>


In [175]:
# Creating a list of image names
images_1_8865 = []
for root, dirs, files in os.walk('C:\\Users\\M246047\\Documents\\Python\\8865\\1'):
    for filename in files:
        image_data = cv2.imread('C:\\Users\\M246047\\Documents\\Python\\8865\\1\\'+filename, 0)
#         print(filename)
#         print(image_data.shape)
        if image_data.shape==(50,50):
            image_data.reshape(1,50,50)
            images_1_8865.append(image_data)
            
images_1_8865 = np.asarray(images_1_8865)
print('There are {} images'.format(len(images_1_8865)))
print('The type of the data is', type(images_1_8865))
images_1_8865_class = np.ones((len(images_1_8865),), dtype=int)

There are 55 images
The type of the data is <class 'numpy.ndarray'>


In [183]:
# Create x_train and y_train
x_train = np.append(images_0_8863, [images_1_8863, images_0_8864, images_1_8864])
y_train = np.append(images_0_8863_class, [images_1_8863_class, images_0_8864_class, images_1_8864_class])

# Create x_test and y_test
x_test = np.append(images_0_8865, images_1_8865)
y_test = np.append(images_0_8865_class, images_1_8865_class)

In [186]:
x_train.shape

(1920003,)

In [206]:
# Create x_train and y_train
print(images_0_8863.shape)
print(images_1_8863.shape)
print(images_0_8863_class.shape)
print(images_1_8863_class.shape)

images_0_8863 = images_0_8863.reshape

x_train = np.append(images_0_8863, images_1_8863)
y_train = np.append(images_0_8863_class, images_1_8863_class)
print(x_train.shape)
print(y_train.shape)

# Create x_test and y_test
x_test = np.append(images_0_8865, images_1_8865)
y_test = np.append(images_0_8865_class, images_1_8865_class)

print('x_test shape:', x_test.shape)
print('y_test shape: ', y_test.shape)

(768, 50, 50)
(207, 50, 50)
(768,)
(207,)
(2437500,)
(975,)
x_test shape: (1780000,)
y_test shape:  (712,)


I think the arrays need to be (X, 2500)
Where is the standardization (/255) coming from?

In [187]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(50, 50)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

ValueError: Input 0 is incompatible with layer conv2d_5: expected ndim=4, found ndim=3

In [ ]:
Loop over directory structure
within each subdirectory, read the files as gray scale
read the array, reshape


In [185]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(50, 50)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

ValueError: Input 0 is incompatible with layer conv2d_4: expected ndim=4, found ndim=3